First we instantiate the simulator class we will use to generate our sequences. The Simulator handles all the particulars about the target being tracked. This simulation in particular is a Linear Time Invariant scenario of an object travelling at a constant velocity in two dimensions. However, it is being observed via a nonlinear measurement model.

In particular it is a Bearing Range sensor positioned at the origin of the coordinate system. Measurements are collected as a 2 dimensional vector containing an angle, and a distance.

In [1]:
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.detection import Detection

from stonesoup.types.track import Track
from stonesoup.types.hypothesis import SingleHypothesis

class simulator():

    def __init__(self, transition_model, measurement_model, state_range, meas_range = None, noise_range = None, meas_noise_range = None):
        """
        Parameters
        ----------
        transition_model : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        measurement_model : :class:`~.Predictor`
            The Updater to be used.
        """
        self.transition_model= transition_model
        self.measurement_model = measurement_model
        self.state_range = state_range
        self.meas_range = meas_range
        self.noise_range = noise_range
        self.meas_noise_range = meas_noise_range

    def _generate_ground_truth(self, prior, time_span):
        
        time_interval = time_span[1]-time_span[0]
        ground_truth = GroundTruthPath([prior])
        for k in range(1, len(time_span)):
            ground_truth.append(GroundTruthState(
                self.transition_model.function(ground_truth[k-1], noise=True, time_interval=time_interval),
                timestamp=time_span[k-1]))
        return ground_truth
    
    def _simulate_measurements(self, ground_truth):
        #Simulate Measurements
        measurements = []
        for state in ground_truth:
            measurement = self.measurement_model.function(state, noise=True)
            measurements.append(Detection(measurement,
                                          timestamp=state.timestamp,
                                          measurement_model=self.measurement_model))
        return measurements

    def _initializer(self, time_stamp):
        state_range_min = self.state_range[0]
        state_range_max = self.state_range[1]
        state_vector = StateVector(np.random.uniform(low=state_range_min, high=state_range_max))
        while np.sqrt(state_vector[1]**2+state_vector[3]**2) < 0.15:
            state_vector = StateVector(np.random.uniform(low=state_range_min, high=state_range_max))
        meas_range_low = self.meas_range[0]
        meas_range_max = self.meas_range[1]
        self.measurement_model.translation_offset = np.random.uniform(low=meas_range_low, high=meas_range_max).reshape(len(meas_range_low),1)
        #process_noise = 
        #self.transition_model. = np.randint(low=0,high=len(noise_range))
        
        return State(state_vector = state_vector, timestamp=time_stamp)

    def generate_training_data(self, time_span):
        """

        Parameters
        ----------
        ground_truth : :class:`~.GroundTruthPath`
            StateMutableSequence type object used to store ground truth.
        initial_state : :class:`~.State`
            Initial state for the ground truth system. This MUST be a State,
            not a State subclass, like GaussianState or EnsembleState.
        prior : :class:`~.GaussianState` or :class:`~.EnsembleState`
            Initial state prediction of tracking algorithm.

        Returns
        -------
        track : :class:`~.Track`
            The Stone Soup track object which contains the list of updated 
            state predictions made by the tracking algorithm employed.
        """

        #Simulate Measurements
        initial_state = self._initializer(time_span[0])
        ground_truth = self._generate_ground_truth(initial_state, time_span)
        measurements = self._simulate_measurements(ground_truth)
        
        return ground_truth, measurements

    def simulate_track(self, predictor, updater, initial_state, prior, time_span):
        """

        Parameters
        ----------
        predictor : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        updater : :class:`~.Predictor`
            The Updater to be used.
        ground_truth : :class:`~.GroundTruthPath`
            StateMutableSequence type object used to store ground truth.
        initial_state : :class:`~.State`
            Initial state for the ground truth system. This MUST be a State,
            not a State subclass, like GaussianState or EnsembleState.
        prior : :class:`~.GaussianState` or :class:`~.EnsembleState`
            Initial state prediction of tracking algorithm.

        Returns
        -------
        track : :class:`~.Track`
            The Stone Soup track object which contains the list of updated 
            state predictions made by the tracking algorithm employed.
        """

        #Simulate Measurements
        ground_truth = self._generate_ground_truth(initial_state, time_span)
        measurements = self._simulate_measurements(ground_truth)
        
        #Initialize Loop Variables
        track = Track()
        for measurement in measurements:
            prediction = predictor.predict(prior, timestamp=measurement.timestamp)
            hypothesis = SingleHypothesis(prediction, measurement)  # Group a prediction and measurement
            posterior = updater.update(hypothesis)
            track.append(posterior)
            prior = track[-1]
        return ground_truth, track

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import datetime

from stonesoup.types.array import StateVector, CovarianceMatrix
from stonesoup.types.state import State, GaussianState

from stonesoup.models.transition.linear import (CombinedLinearGaussianTransitionModel,
                                                ConstantVelocity)
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.updater.kalman import KalmanUpdater, ExtendedKalmanUpdater
from stonesoup.predictor.ensemble import EnsemblePredictor
from stonesoup.predictor.kalman import KalmanPredictor, ExtendedKalmanPredictor
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange

"""
    This script represents the code used to gather the data used in [PAPER HERE].
    
    This repository is structured such that different stone soup algorithms 
    can be run rapidly. Hopefully I've made it modular enough to 
    allow swapping of things like algorithms, and "experiments" by replacing
    the desired transition and measurement models.
    
    The simulator class requires a transition and 
    measurement model, then the simulate_track method accepts a Stone Soup
    Predictor, Updater, ground truth initial state, initial state for the
    chosen algorithm, and a span of time which the simulation takes place over.
    This time span should be an evenly spaced datetime.datetime list.
    
    The simulator then, is used to gather "Track" instances, and with a list 
    of tracks, RMSE can then be calculated.
"""

i = 60
num_vectors = i*5

"""
    Here, we get our initial variables for simulation. For this, we are just
    using a time span of 60 time instances spaced one second apart.
"""

timestamp = datetime.datetime(2021, 4, 2, 12, 28, 57)
tMax = 5
dt = 1
tRange = tMax // dt
plot_time_span = np.array([dt*i for i in range(tRange)])

time_span = np.array([timestamp + datetime.timedelta(seconds=dt*i) for i in range(tRange)])

"""
Here we instantiate our transition and measurement models. These are the 
same models used in the StoneSoup Kalman Filter examples.
"""

q_x = 0.05
q_y = 0.05
sensor_x = 0  # Placing the sensor off-centre
sensor_y = 0

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

measurement_model = CartesianToBearingRange(
ndim_state=4,
mapping=(0, 2),
noise_covar=np.diag([np.radians(1)**2, 0.1**2]),  # Covariance matrix. 0.2 degree variance in
# bearing and 1 metre in range
translation_offset=np.array([[sensor_x], [sensor_y]])  # Offset measurements to location of
# sensor in cartesian.
)


In order to use this simulated data, we need to format it such that it can be loaded into PyTorch easily. Pytorch requires us to specify functions "__get_item__" and "__len__" in order to be usable by the generic Data Loader. I simply included the init and a helper function to append trajectories to the dataset for ease of use.

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np

# Datasets should be specified on a per simulation basis

#class sequence_generator(Dataset):

class dataset_2D_bearing_range(Dataset):
    #Range Bearing 2D Dataset Sequence Packer

    def __init__(self, dataset, ground_truth=None, measurements=None):
        if dataset == None:   
            gt = np.array([e.state_vector for e in ground_truth]).squeeze().T
            gt = torch.tensor(gt.astype(dtype=np.float32),device = device)
            ms = np.array([m.state_vector for m in measurements]).squeeze().T
            ms = torch.tensor(ms.astype(dtype=np.float32),device = device)
            self.dataset = torch.unsqueeze(torch.cat((ms,gt)),dim=0)
        else:
            self.dataset = dataset
            
    
    def append_to_dataset(self, ground_truth, measurements):
        """
        Parameters
        ----------
        ground_truth : :class:`~.list`
            A list of Stonesoup States.
        measurements : :class:`~.measurements`
            The list of Stonesoup Measurements to be used.
        """
    
        gt = np.array([e.state_vector for e in ground_truth]).squeeze().T
        gt = torch.tensor(gt.astype(dtype=np.float32),device = device)
        ms = np.array([m.state_vector for m in measurements]).squeeze().T
        ms = torch.tensor(ms.astype(dtype=np.float32),device = device)
        new_entry = torch.cat((ms,gt),dim=0)

        self.dataset = torch.cat((self.dataset,torch.unsqueeze(new_entry,dim=0)),dim=0)

    def __getitem__(self, idx):
        return self.dataset[idx,0:2].T, self.dataset[idx,2:6].T

    def __len__(self):
        return self.dataset.shape[0]



This next module is something I call the Composer. The Composer's job is to return a useful dataset for use in PyTorch, and I will tell it how many trajectories I want, and it will call the relevant simulator and dataset methods to generate a dataset for the purpose of learning.

In [4]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np

# Datasets should be specified on a per simulation basis
# Should look up realistic values for radar precision and initial states
# Come up with realistic constraints for initial velocities

#class sequence_generator(Dataset):

class dataset_composer():
    #Range Bearing 2D Dataset Sequence Packer
    def __init__(self, simulators, training_dataset, testing_dataset, batch_size, num_workers):
        #pass a list of simulators
        self.simulators = simulators
        self.training_dataset = training_dataset
        self.testing_dataset = testing_dataset
        self.batch_size = batch_size
        self.num_workers = num_workers

    def simulate_trajectories(self, training_dataset_len, testing_ratio):
        for simulator in self.simulators:
            trainset_length = int((training_dataset_len-1) / len(self.simulators)) # Partition dataset into sets from each simulator
            for i in range(trainset_length):
                ground_truth, measurements = simulator.generate_training_data(time_span)
                self.training_dataset.append_to_dataset(ground_truth, measurements)
            trainloader = DataLoader(self.training_dataset)

            testset_length = int(((training_dataset_len)*testing_ratio -1)/len(self.simulators))
            for i in range(testset_length):
                ground_truth, measurements = simulator.generate_training_data(time_span)
                self.testing_dataset.append_to_dataset(ground_truth, measurements)
            testloader = DataLoader(self.testing_dataset)
        return trainloader, testloader

    def output_to_file(self, trainset_name, testset_name):
        torch.save(self.training_dataset, trainset_name)
        torch.save(self.testing_dataset, testset_name)

    def read_from_file(self, training_file_name, testing_file_name, device):
        trn = torch.load(training_file_name, map_location = device)
        tst = torch.load(testing_file_name, map_location = device)
        return DataLoader(trn, batch_size = self.batch_size, num_workers=self.num_workers), DataLoader(tst, batch_size = self.batch_size, num_workers=self.num_workers)




Here we instantiate the simulator, provide a range of permissable values for the initial conditions, noise coefficients, and specify the transition model, and measurement model.



In [5]:
# Position, Velocity, Position, Velocity
state_range_min = np.array([-15, -2, -15, -2])
state_range_max = np.array([15, 2, 15, 2])

# Add some sort of miniumum speed

meas_range_min = np.array([0, 0])
meas_range_max = np.array([0, 0])

# Randomization for process noise
#process_noise_coefficients = [0.005, 0.05, 0.5]
process_noise_coefficients = [0.05]

nonlinear_simulator = simulator(transition_model=transition_model,
                      measurement_model=measurement_model,
                      state_range = (state_range_min, state_range_max),
                      meas_range = (meas_range_min, meas_range_max)
)

ground_truth, measurements = nonlinear_simulator.generate_training_data(time_span)
training_dataset =  dataset_2D_bearing_range(dataset = None, ground_truth = ground_truth, measurements = measurements)
testing_dataset =  dataset_2D_bearing_range(dataset = None, ground_truth = ground_truth, measurements = measurements)

#training_dataset_len = 1000000 #  1,000,000
training_dataset_len = 100000 #  100,000
#training_dataset_len = 10000 # 10,000
testing_ratio = 0.15

batch_size = 128
num_workers = 0

dataset_composer = dataset_composer([nonlinear_simulator], testing_dataset, training_dataset, batch_size = batch_size, num_workers = num_workers)

trainloader, testloader = dataset_composer.simulate_trajectories(training_dataset_len, testing_ratio)
dataset_composer.output_to_file('training_data','testing_data')
#trainloader, testloader = dataset_composer.read_from_file('training_data', 'testing_data', device)
#print('Training Dataset Length:', training_dataset.__len__())
#print('Testing Dataset Length:',testing_dataset.__len__())


Turn Sequences into dataset